# Hands-on: watsonx.ai & Watson Discovery 사용한 RAG(Retrieval-Augmented Generation) 애플리케이션 만들기


## Overview
이 Jupyter Notebook은 다음 방법에 대한 예를 제공합니다.

1. Watson Discovery 컬렉션을 생성하고 여기에 문서를 업로드합니다.

2. 간단한 RAG 연습을 수행하려면 이 노트북을 사용자 정의하세요.

프롬프트/쿼리는 이 노트북을 통해 전달됩니다. 코드는 Watson Discovery 컬렉션의 문서에서 **검색** 작업을 수행합니다. 결과를 생성하기 위해 선택한 LLM(대형 언어 모델)에 대한 프롬프트와 함께 반환된 정보입니다.

In [ ]:
# Install library
!pip install --upgrade ibm-watson

In [ ]:
# Import libraries
import json
import os

from ibm_watson import DiscoveryV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# WML python SDK
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods

## 1. Watson Discovery 설정

Watson Discovery를 설정할 때 **ibm-credentials.env**라는 파일에 인증 정보를 저장해야 합니다. 해당 파일의 값을 사용해야 합니다. 간단한 텍스트 편집기를 사용하여 파일을 열 수 있습니다.

1. 파일에서 **DISCOVERY_APIKEY** 값을 찾아 아래 **IAMAuthenticator** 값으로 붙여넣습니다(작은따옴표 2개 사이).

2. 파일에서 **DISCOVERY_URL** 값을 찾아 아래의 **discovery.set_service_url** 값으로 붙여넣습니다(작은따옴표 2개 사이).

이렇게 하면 미리 로드된 PDF 문서(IBM 연례 보고서 2022)를 사용하여 Watson Discovery 인스턴스에 대한 연결이 초기화됩니다.

In [ ]:
#Set up Watson Discovery credentials
authenticator = IAMAuthenticator('<YOUR WATSON DISCOVERY API KEY HERE>') # DISCOVERY_APIKEY  
discovery = DiscoveryV2(
    version='2020-08-30',
    authenticator=authenticator
)

discovery.set_service_url('<YOUR WATSON DISCOVERY URL HERE>') # DISCOVERY_URL

wd_project_id = '<YOUR PROJECT ID HERE>'            # Watson Discovery Project ID
# wd_collection_ids = ['<YOUR COLLECTION ID HERE>']   # Watson Discovery Collection IDs

컬렉션 명으로 컬렉션 아이디 찾기

In [ ]:
def find_collection_id(collection_ids, collection_name):
    # Loop through each collection in the collections list
    for collection in collection_ids['collections']:
        # Check if the current collection's name matches the provided collection_name
        if collection['name'] == collection_name:
            # If a match is found, return the collection_id
            return collection['collection_id']
    # If no match is found after checking all collections, return None
    return None

In [ ]:
collection_ids_result = discovery.list_collections(
    project_id=wd_project_id
).get_result()

print(collection_ids_result)

In [ ]:
collection_name = '<YOUR COLLECTION NAME>'
wd_collection_ids = [find_collection_id(collection_ids_result, collection_name)]
print(f"The collection ID for '{collection_name}' is: {wd_collection_ids}")

WD 조회 함수 생성

In [ ]:
def WD_search(question):
    # Utilize the IBM Watson Discovery service to query a collection for information based on a natural language query
    response = discovery.query(
  project_id=wd_project_id,
  collection_ids = wd_collection_ids,
    passages = {'enabled': True, 
                'max_per_document': 5,
                'find_answers': True},
    natural_language_query = question
    ).get_result()


## 2. Watson Discovery Search 

이는 모델에 게시되는 간단한 질문(프롬프트)입니다. 이는 Streamlit GUI에서 드러날 수 있습니다. 이는 실습의 초점이 아닙니다. 클라이언트에는 다른 GUI 도구가 있을 수 있습니다. 여기서는 기본 Watson Discovery에 초점을 맞추고 나중에는 watsonx.ai에 중점을 둡니다.

In [ ]:
question = '기업의 생성형AI 도입 현황은?'

아래 블록의 경우 생성한 Watson Discovery 프로젝트에서 적절한 정보를 제공해야 합니다.

1. **프로젝트 ID**에 아래 **project_id** 값을 작은따옴표 2개 사이에 붙여넣습니다.

2. **컬렉션 ID**(예:IBM 연차 보고서 2022 보고서가 포함된 컬렉션의 경우) 아래 **collection_ids** 값을 작은따옴표 2개 사이에 붙여넣습니다.

Watson Discovery Search에 대해 정의된 몇 가지 매개변수가 있습니다.

* **passages.enabled**: 서비스가 natural_언어_query 매개변수를 사용하는 쿼리에 의해 반환된 문서에서 가장 관련성이 높은 구절 집합을 반환하는지 여부를 지정하는 부울입니다. Watson Discovery는 정교한 알고리즘을 사용하여 쿼리에서 반환된 모든 문서에서 가장 적합한 텍스트 구절을 결정합니다. 각 문서 쿼리 결과 내의 섹션으로 표시되며 구절 관련성에 따라 정렬됩니다. 쿼리에 구절 검색을 포함하면 구절을 채점하는 데 더 많은 시간이 걸리기 때문에 응답 시간이 늘어납니다.

* **passages.max_per_document**: 기본적으로 문서당 하나의 구절이 반환됩니다. Passages.max_per_document 매개변수에 더 높은 숫자를 지정하여 문서당 반환할 최대 구절 수를 늘릴 수 있습니다.

* **find_answers**: 기본적으로 Watson Discovery는 자연어 쿼리에 대한 답변이 포함된 전체 구절을 반환하여 답변을 제공합니다. 답변 찾기 기능이 활성화되면 Watson Discovery는 구절 내에서 "단답"을 제공하고, "단답"이 사용자 쿼리에서 명시적이거나 암시적인 질문에 답변하는지 여부를 표시하는 신뢰도 점수도 제공합니다.

* **natural_언어_query**: IBM Watson Assistant와 같은 대화형 또는 자유 텍스트 인터페이스에서 사용자로부터 수신될 수 있는 것처럼 자연어로 표현된 쿼리를 입력하려면 자연어 쿼리를 사용합니다. 매개변수는 전체 입력을 쿼리 텍스트로 사용합니다. 연산자를 인식하지 못합니다. 자연어 쿼리의 최대 쿼리 문자열 길이는 2048입니다.
   
쿼리 매개변수에 대한 자세한 내용은 https://cloud.ibm.com/docs/discovery-data?topic=discovery-data-query-parameters를 참조하세요.

In [ ]:
# Utilize the IBM Watson Discovery service to query a collection for information based on a natural language query
response = discovery.query(
  project_id=wd_project_id,
  collection_ids = wd_collection_ids,
  passages = {'enabled': True, 
              'max_per_document': 5,
             'find_answers': True},
  natural_language_query = question
).get_result()

with open('data.json', 'w') as f:
    json.dump(response, f)

다음 4개 블록은 출력에 대한 일부 구문 분석을 제공합니다. 이를 업데이트할 필요는 없습니다.

In [ ]:
# Inspecting the key fields in the WD output
response.keys()

In [ ]:
# Only one relevant document (because one document in the collection)
len(response['results'])

In [ ]:
# Removing some tags
passages = response['results'][0]['document_passages']
passages = [p['passage_text'].replace('<em>', '').replace('</em>', '').replace('\n','') for p in passages]
passages

In [ ]:
# Concatenating passages
context = '\n '.join(passages)
context

함수로 선언

In [ ]:
def WD_search(question):
    # Utilize the IBM Watson Discovery service to query a collection for information based on a natural language query
    response = discovery.query(
    project_id=wd_project_id,
    collection_ids = wd_collection_ids,

    passages = {'enabled': True, 
                'max_per_document': 5,
                'find_answers': True},
    natural_language_query = question
    ).get_result()

    passages = response['results'][0]['document_passages']
    passages = [p['passage_text'].replace('<em>', '').replace('</em>', '').replace('\n','') for p in passages]
    context = '\n '.join(passages)
    return context

## 3. Creating Prompt

이 섹션에서는 LLM이 Watson Discovery에서 검색한 구절과 아래 지정된 규칙을 기반으로 답변을 생성할 수 있도록 지침과 컨텍스트가 포함된 프롬프트를 생성합니다.

In [ ]:
# https://huggingface.co/blog/llama2#how-to-prompt-llama-2
def make_prompt(context,question) :
    prompt = f"""
    가이드:
    다음 질문에 문서 내용을 참조하여 답변하세요.
    답변은 반드시 한국어를 사용 하세요. 한자어가 포함된 경우 한글로 반드시 변경하세요.
    질문에 답할 수 없으면 '답변할 수 없음'이라고 말하세요.
    질문에 답변했다면 '답변할 수 없음'이라고 말하지 마세요.
    질문과 관련이 없는 정보는 포함하지 마세요.
    다른 질문에는 대답하지 마십시오.
    반복을 사용하지 마십시오.

    문서: '''{context}'''
    질문: '''{question}''' 
    답변:


    """
    # complete_prompt = context + instruction + question

#     print("----------------------------------------------------------------------------------------------------")
#     print("*** Prompt:" + prompt + "***")
#     print("----------------------------------------------------------------------------------------------------")
    return prompt

In [ ]:
prompt = make_prompt(context,question)
print(prompt)

## 4. Configuring watsonx.ai

다음 섹션에서는 LLM(대형 언어 모델)에 대한 입력을 정의합니다. 지정해야 하는 유일한 항목은 watsonx.ai의 project_id입니다. 값을 **project_id**(2개의 큰따옴표 사이)에 붙여넣습니다.



In [ ]:
# Initialize the watsonx model
def get_model(watsonx_project_id,model_type,max_tokens,min_tokens,decoding,temperature):#, repetition_penalty):

    generate_params = {
        GenParams.MAX_NEW_TOKENS: max_tokens,
        GenParams.MIN_NEW_TOKENS: min_tokens,
        GenParams.DECODING_METHOD: decoding,
        GenParams.TEMPERATURE: temperature,
    }
    
    model = Model(
        model_id=model_type,
        params=generate_params,
        credentials={
            "apikey": api_key,
            "url": url
        },
        project_id= watsonx_project_id
        )

    return model

이 섹션에서는 watsonx.ai에 대한 자격 증명을 제공합니다.

1. watsonx.ai **프로젝트 ID**(Waston Discovery의 ID 아님) 값을 **watsonx_project_id**(2개의 큰따옴표 사이)에 붙여넣습니다.

2. **API 키**(Watson Discovery용 키 아님) 값을 **api_key**(2개의 큰따옴표 사이)에 붙여넣습니다.

In [ ]:
# URL of the hosted LLMs is hardcoded because at this time all LLMs share the same endpoint
url = "https://us-south.ml.cloud.ibm.com"

# Replace with your watsonx project id (look up in the project Manage tab)
watsonx_project_id = "<YOUR WATSONX.AI PROJECT ID HERE>"

# Replace with your IBM Cloud key
api_key = "<YOUR IBM CLOUD API KEY HERE>"

다음 블록은 LLM의 세부 사항을 지정합니다. PoX에서는 이러한 값을 다양하게 변경하여 클라이언트에게 최상의 결과를 얻을 수 있는 방법을 보여줄 수 있습니다.

1. **model_type**은 사용 중인 LLM을 지정합니다. 아래 예에서는 llama-2-70b-chat 모델입니다. 다른 모델로 변경하실 수 있습니다. 모델의 크기는 리소스 사용량에 영향을 미칩니다. PoX에서 다른 방법을 시도해 보고 다른 결과가 나오는지 확인할 수도 있습니다. 아래 블록에는 4개의 모델이 있습니다(3개가 주석 처리되었으므로 llama2가 사용됩니다. 다른 모델을 주석 처리하여 시도해 볼 수 있습니다).

2. **max_tokens**는 출력 토큰의 최대 수를 지정합니다. 토큰 1개가 단어 1개와 같지 않다는 점을 명심하세요. 일반적으로 단어당 대략 3개의 토큰을 추정할 수 있습니다.

3. **min_tokens**는 출력 토큰의 최소 수를 지정합니다.

4. **디코딩**은 디코딩 방법을 지정합니다. **샘플링** 디코딩을 수행하도록 선택할 수도 있습니다. 이 경우 더 많은 매개변수(예: **Top-P** 및 **Top-K**)를 지정할 수 있습니다. 이러한 추가 매개변수에 대한 자세한 내용은 Watsonx.ai 기술 영업 레벨 3 수업(https://learn.ibm.com/course/view.php?id=13452)에서 확인할 수 있습니다.

5. **온도**는 모델이 얼마나 보수적이거나 창의적인지를 지정합니다. 낮을수록 보수적입니다. 범위는 0~2입니다.

In [ ]:
# Set up watsonx model and parameters
model_type = "meta-llama/llama-3-70b-instruct"
# model_type = "meta-llama/llama-2-70b-chat"
# model_type = "google/flan-t5-xxl"
# model_type = "ibm/granite-13b-chat-v1"
# model_type = "ibm/granite-13b-instruct-v1"
# model_id = "ibm/mpt-7b-instruct2"
max_tokens = 500
min_tokens = 50
decoding = DecodingMethods.GREEDY
temperature = 0.7

# Get the watsonx model
model = get_model(watsonx_project_id,model_type, max_tokens, min_tokens, decoding, temperature)

## 5. Answer Generation

이 블록은 입력 프롬프트, 지정된 매개변수, 그리고 무엇보다도 지정된 Watson Discovery 데이터 컬렉션을 기반으로 답변을 생성합니다.

In [ ]:
# Send a prompt to model
def watsonx_call(prompt):
    generated_response = model.generate(prompt)
    response_text = generated_response['results'][0]['generated_text']

    # Print model response
    print("--------------------------------- Generated response -----------------------------------")
    print(response_text)
    print("*********************************************************************************************")

In [ ]:
watsonx_call(prompt)

In [ ]:
question = '기업의 생성형AI 도입 현황은?'
# question = "기업에서 생성형 AI를 도입하는 가치는?"
# question = "기업에서 생성형 AI를 도입하기 위해 필요한 고려사항은?"
# question = "기업에서 생성형 AI 도입을 가로막는 가장 큰 장애물은?"

In [ ]:

context = WD_search(question)

In [ ]:
prompt = make_prompt(context,question)
watsonx_call(prompt)